# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

I will be using a dataset of 300.000 rows with different posts to determine the following:
* Do Ask HN or Show HN receive more comments on average?
* Do posts created at a certain time receive more comments on average?

Let's start by importing the libraries we need and reading the data set into a list of lists.

In [3]:
from csv import reader

opened_file = open('HN_posts_year_to_Sep_26_2016.csv', encoding="utf8")
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[0:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16'], ['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']]


Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles. To find the posts that begin with either Ask HN or Show HN, we'll use the string method startswith.

In [13]:
ask_posts = []
show_posts = []
other_posts = []

for each_row in hn:
    name = each_row[1]
    if name.lower().startswith('ask'):
        ask_posts.append(name)
    elif name.lower().startswith('show'):
        show_posts.append(name)
    else:
        other_posts.append(name)
        
print('Ask posts: ', len(ask_posts))
print(ask_posts[:5])
print('\n')
print('Show posts: ',len(show_posts))
print(show_posts[:5])
print('\n')
print('Other posts: ',len(other_posts))
print(other_posts[:5])

Ask posts:  9269
['Ask HN: What TLD do you use for local development?', 'Ask HN: How do you pass on your work when you die?', 'Ask HN: How a DNS problem can be limited to a geographic region?', 'Ask HN: Why join a fund when you can be an angel?', 'Ask HN: Someone uses stock trading as passive income?']


Show posts:  10218
['Show HN: Finding puns computationally', 'Show HN: A simple library for complicated animations', 'Show HN: WebGL visualization of DNA sequences', 'Show HN: Pomodoro-centric, heirarchical project management with ES6 modules', 'Show HN: Jumble  Essays on the go #PaulInYourPocket']


Other posts:  273632
['You have two days to comment if you want stem cells to be classified as your own', 'SQLAR  the SQLite Archiver', 'What if we just printed a flatscreen television on the side of our boxes?', 'algorithmic music', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake']


Next, let's determine if ask posts or show posts receive more comments on average, both per post (number of comments / number of this type's post) and in total (number of comments / total number of comments).

In [14]:
total_ask_comments = 0
total_show_comments = 0
total_rest_comments = 0
total_comments = 0

for each_row in hn:
    name = each_row[1]
    n_comments = int(each_row[4])
    total_comments += n_comments
    if name.lower().startswith('ask'):
        total_ask_comments += n_comments
    elif name.lower().startswith('show'):
        total_show_comments += n_comments

avg_ask_comments = round(total_ask_comments/len(ask_posts),2)
avg_show_comments = round(total_show_comments/len(show_posts),2)

avg_ask_total = round((total_ask_comments/total_comments)*100,2)
avg_show_total = round((total_show_comments/total_comments)*100,2)

print('N ask comments: ', total_ask_comments, ', average per post: ', avg_ask_comments, ', overall average: ', avg_ask_total)
print('N show comments: ', total_show_comments, ', average per post: ', avg_show_comments, ', overall average: ', avg_show_total )

N ask comments:  95736 , average per post:  10.33 , overall average:  5.01
N show comments:  49684 , average per post:  4.86 , overall average:  2.6


So, do show posts or ask posts receive more comments on average? The answer would be, yes, ask comments receive more comments per post. Also, it is worth mentioning that the "overall averages" shown here only sum up to 7.61%, so 92.39% of the comments are distributed on the rest of the posts. 

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

* Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
* Calculate the average number of comments ask posts receive by hour created.

And to do so, we created two dictionaries:

* counts_by_hour: contains the number of ask posts created during each hour of the day.
* comments_by_hour: contains the corresponding number of comments ask posts created at each hour received.

In [36]:
import datetime as dt

result_list = []
counts_by_hour = {}
comments_by_hour = {}

ask_post_full = []

for each_row in hn:
    name = each_row[1]
    if name.lower().startswith('ask'):
        ask_post_full.append(each_row)

for each_row in ask_post_full:
    created_at = each_row[6]
    n_comments = each_row[4]
    result_list.append([created_at, n_comments])

for row in result_list:
    comment_number = int(row[1])
    date = row[0]
    date = dt.datetime.strptime(date, '%m/%d/%Y %H:%M')
    date = dt.datetime.strftime(date,'%H:%M')
    if date in counts_by_hour:
        counts_by_hour[date] +=1
        comments_by_hour[date] += comment_number
    else:
        counts_by_hour[date] = 1
        comments_by_hour[date] = comment_number
        

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [60]:
avg_by_hour = []

for each_hour in comments_by_hour:
    if each_hour in counts_by_hour:
        calc_avg = comments_by_hour[each_hour]/counts_by_hour[each_hour]
        avg_by_hour.append([calc_avg, each_hour])

print(avg_by_hour[:8])
        

[[3.2857142857142856, '02:53'], [6.2727272727272725, '01:17'], [10.25, '22:57'], [5.8, '22:48'], [2.4, '21:50'], [3.0, '19:30'], [4.428571428571429, '19:22'], [4.0, '17:55']]


Obviously, this might seem confusing at first, as we have the average in the first slot and the time in the second slot, but this makes sense now to order it, so we can swap those two values afterwards.

In [75]:
sort_avg_by_hour = sorted(avg_by_hour, reverse=True)

swap_avg_by_hour = []

for each_row in sort_avg_by_hour:
    hour = each_row[1]
    avg = each_row[0]
    swap_avg_by_hour.append([hour, avg])

print("Top 5 Hours for Ask Posts Comments")

count = 0 
for each_row in swap_avg_by_hour:
    hour = each_row[0]
    avg = each_row[1]
    template = "{0}: {1:.2f} average comments per post".format(hour, avg)
    print(template)
    count +=1
    if count == 5:
        break


Top 5 Hours for Ask Posts Comments
04:29: 660.00 average comments per post
15:01: 246.51 average comments per post
15:00: 177.27 average comments per post
02:07: 155.50 average comments per post
05:25: 123.75 average comments per post


Based on this findings, and considering that the "created_at" column is the date and time the post was made, being the time zone Eastern Time in the US, if I, currently living in Spain, would like to maximaze the number of comments received after posting in Hacker News, I would need to post in the following hours:

In [87]:
print("Top 5 Hours for Ask Posts Comments (Spain's time zone):")

count = 0 
for each_row in swap_avg_by_hour:
    prov_date = dt.datetime.strptime(each_row[0],"%H:%M")
    hour = prov_date + dt.timedelta(hours=6)
    hour = dt.datetime.strftime(hour, "%H:%M")
    avg = each_row[1]
    template = "{0}: {1:.2f} average comments per post".format(hour, avg)
    print(template)
    count +=1
    if count == 5:
        break

Top 5 Hours for Ask Posts Comments (Spain's time zone):
10:29: 660.00 average comments per post
21:01: 246.51 average comments per post
21:00: 177.27 average comments per post
08:07: 155.50 average comments per post
11:25: 123.75 average comments per post
